# Step 1:  Create supply (runoff)
This script summarizes the PCR-GLOBWB NetCDF outputs for runoff by the Hydrobasin 6 catchments. We loop through the GCM/scen, resample by 5x5, convert the flux to volume, and take a clip of the timeseries data by each polygon. 

## Runoff (renewable water available inside sub-basin. Internal consumption is not removed)
runoff = land_surface_runoff * area

## After this script:
The results will need to be added with discharge. The results have been resampled and will need to be adjusted.

# Setup

## Libraries

In [0]:
!pip install tqdm
!pip install rtree
!pip3 install numpy
!pip3 install pandas
!pip3 install scipy
!pip3 install geopandas
!pip3 install xarray
!pip3 install rasterio
!pip3 install rasterstats
!pip3 install rioxarray
!pip3 install netcdf4
!pip install psutil
!pip install dask
import psutil
import xarray
import rioxarray
import rasterio
import geopandas as gpd
import rasterstats as rstats
import netCDF4, os, subprocess, re, time, datetime, json
import numpy as np, pandas as pd
import netCDF4 as nc
from rasterio import Affine
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import dask
import gc
from joblib import Parallel, delayed


Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (4.64.1)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: rtree in /databricks/python3/lib/python3.8/site-packages (1.0.0)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (1.19.2)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (1.2.4)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas) (1.15.0)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (1.6.2)
Requirement already satisfied: numpy<1.23.0,>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from scipy) (1.19.2)
 WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geopandas in /databricks/python3/lib/python3.8/site-packages (0.11.1)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from geopandas) (20.9)
Requirement already satisfied: pyproj>=2.6.1.post1 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (3.3.1)
Requirement already satisfied: fiona>=1.8 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.21)
Requirement already satisfied: shapely<2,>=1.7 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.4)
Requirement already satisfied: pandas>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.2.4)
Requirement already satisfied: munch in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2.5.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from fiona>=1.8->geopandas) (52.0.0)
Requirement already satisfied: click>=4.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (8.1.3)
Requirement already satisfied: cligj>=0.5 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (0.7.2)
Requirement already satisfied: six>=1.7 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.15.0)
Requirement already satisfied: attrs>=17 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (20.3.0)
Requirement already satisfied: click-plugins>=1.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.1.1)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2020.12.5)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (2020.5)
Requirement alrea

## Functions & Data Locations

In [0]:
def memory_usage():
    process = psutil.Process(os.getpid())
    print('- - - Current memory usage is:', process.memory_info()[0] / float(2 ** 20))
    

def find_runoff_paths(gcm, scen, m):
    '''
    PURPOSE: Find the paths to the supply data based on the defined parameters
    INPUTS:
        gcm: global climate model
        scen: future scenario
        m: region of the world
    OUTPUTS:
        dPATH: path to 5 arc min discharge
    '''
    # Find withdrawal data paths
    # For withdrawal data, Path requires knowing the beginning year
    beginText = 'begin_from_1960' if 'historical' in scen else 'begin_from_2015'
    scenText = 'historical-reference' if gcm == 'gswp3-w5e5' else scen
    # Find runoff
    rPATH = '/dbfs/mnt/pgb-data-lake/pcrglobwb_output1/pcrglobwb_aqueduct_2021/version_2021-09-16/{0}/{1}/{2}/{3}/netcdf/runoff_monthTot_output.nc'.format(gcm, scenText, beginText, m)
    return rPATH

def read_NETCDF(ncPATH):
    '''
    PURPOSE: Read in NetCDF, return an Xarray about with spatial dimension defined, and list of band names (so we know what data is in it)
    INPUTS:
        ncPATH: path to netCDF in Data Lake
    OUTPUTS:
        ds: Xarray
        nc_band: band in data
    '''
#     print(ncPATH)
    # Read in arrary
    ds = xarray.open_dataset(ncPATH)
    # Find coordinate names
    dimensions  = [x for x in ds.coords.keys()]
    lat_variable = [x for x in dimensions if "lat" in x][0]
    lon_variable = [x for x in dimensions if "lon" in x][0]
    # Standardize lat and lon names
    ds = ds.rename({lon_variable: 'lon', lat_variable: 'lat'})
    # Set spatial dimenstions and projection
    ds = ds.rio.set_spatial_dims('lon', 'lat')
    ds.rio.crs
    ds.rio.write_crs("epsg:4326", inplace=True)
    # Find name of bands
    nc_bands = list(set([x for x in ds.variables.keys()]) - set(dimensions))
    nc_bands.remove('spatial_ref')
#     print(nc_bands)
    return ds

def fillnas(da):
    """Replaces NA values with 0 in data array. Returns data array"""
        # Fill NA's with where statement. fillna functions aren't working great
    da_filled = xarray.where(da.isnull(), 0, da)
    del da
    # reset spatial dimensions
    da_filled = da_filled.rio.set_spatial_dims('lon', 'lat')
    da_filled.rio.crs
    da_filled.rio.write_crs("epsg:4326", inplace=True)
    return da_filled

def resample_xarray(ds, downscale_factor):
    '''
    PURPOSE: Resample NetCDF to smaller size so zonal statistics can be more accurate 
    INPUTS:
        ds: Xarray to downscale
        downscale_factor: 1-dimensional factor to increase size by. 
        Ex: 10 would turn each pixel into 100 smaller, identical pixels (10X10)
    OUTPUTS:
        xds_downscaled: downscaled Xarray
    '''
    # Dfein new dimensions
    new_width = ds.rio.width * downscale_factor
    new_height = ds.rio.height * downscale_factor
    # Run resampling function
    xds_downscaled = ds.rio.reproject(
        ds.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.nearest,
    )
    # Rename coordinate dimensions
    xds_downscaled = xds_downscaled.rename({'x': 'lon', 'y': 'lat'})
    return xds_downscaled

def flux_to_volume(ds, ds_band):
    '''
    PURPOSE: Convert flux (m/month) to volume (m3/month)
    INPUTS:
        ds: Xarray in flux (m/month) to convert
        ds_band: band name of flux data
    OUTPUTS:
        ds_vol: 5 arc min dataset, band = 'volume'
    '''
    print("- - - Conversion to volume (million m3) started")
    memory_usage()
    # 1. Clip area to region
    ds_area_c = ds_area.sel(lon=slice(ds['lon'].min(),ds['lon'].max()), lat=slice(ds['lat'].max(), ds['lat'].min()))
    print("- - - - total area (m2):", ds_area_c[area_band].sum().values)
    # 2. Turn area into million m2. Do it now instead on PCR data later because it makes messy no data values
    ds_area_c = ds_area_c.assign(variable=ds_area_c[area_band]/1000000.0)
    print("- - - - total area (Million m2):", ds_area_c['variable'].sum().values)
    # 3. Reset Area's lat and lon to match pcr data (i.e. match truncated coords to untruncated)
    gridarea = ds_area_c.reindex_like(ds, method='nearest', tolerance=0.01)
    del ds_area_c
    # 4. Merge data into 1 dataset, Multiply area (million m2) times flux (m)
    ds_adj = xarray.merge([ds, gridarea['variable']])
    ds_adj = ds_adj.assign(volume=ds_adj[ds_band] * ds_adj['variable'])
    del ds
    print("- - - - total WW in Million m3:", ds_adj['volume'].sum().values)
    print("- - - Conversion complete.")
    memory_usage()
    return ds_adj

def segment_id_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# ! - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - - UNIVERSAL DATA - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - -  - - !
# 1. M folders (ie, regional folders) and path to example regional data
mFolders = ['M' + str(x).zfill(7) for x in range(1, 54)]
# Global climate models
# gcmFolders = ['gswp3-w5e5']

gcmFolders = ['gfdl-esm4',
              'ipsl-cm6a-lr',
              'mpi-esm1-2-hr',
              'mri-esm2-0',
              'ukesm1-0-ll']
# Future scenarions
# scenFolders = ['historical',
#                'ssp126',
#                'ssp370',
#                'ssp585']
scenFolders = ['ssp585']
# 2. Hydrobasin 6 
shapePATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/hydro_basin_lv6/aq3_pfaf_basins.shp'
hy6 = gpd.read_file(shapePATH , crs="epsg:4326")
hy6.columns= hy6.columns.str.lower()
project_crs = hy6.crs # WGS84 aka epsg 4326
hy6.set_index('pfaf_id', inplace = True)
geog_id = 'pfaf_id'

# 3. Area
areaPATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/global_area_5arcmin.nc'
ds_area = read_NETCDF(areaPATH)
area_band = 'global_cellsize_m2_05min.tif'  
    

# # 4. Output ( 0 = resample; 1 = GCM; 2 = SCEN; 3 = M region; )
# outPATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/pcrglobwb_aqueduct_2021/version_2021-09-16/run_202205/zonal_statistics/pfaf6/runoff_resample_{0}/{1}_{2}_{3}.csv'.format

# 4. Output {root: 0 = GCM; 1 = SCEN, name: 0 = m region}
newROOT = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/pcrglobwb_aqueduct_2021/version_2021-09-16/run_202205/zonal_statistics/pfaf6/runoff_resample_{0}/{1}/{2}/'.format
newNAME = '{0}.csv'.format

In [0]:
def run_runoff_parallelized(gcm, scen, m, resample_size):
    '''
    PURPOSE: Find the mean runoff
    INPUTS:
        gcm: global climate model
        scen: scenario
        m: region

    OUTPUTS:
        None returned. CSV containing zonal statistics will save. 
    '''
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - # 
    # In function function
    def clip_by_pfaf_id(p):
        # Step 3.1: Select 1 polygon per loop
        my_geom = hy6.loc[p:p, :]
        # Step 3.2: Clip NetCDF by polygon
        stime = time.time()
        clipped = rechunked2.rio.clip(my_geom.geometry, project_crs, drop=False)
        # Step 3.2 housekeeping
        print('- - - - Clipped NetCDF in {}'.format(time.time()-stime))
        memory_usage()
        # Step 3.3: Sum contents across lat and long
        geom_sum = clipped.sum(dim = ['lon', 'lat'])
        # Step 3.3 housekeeping
        del clipped
        # Convert array to Series
        df_t = geom_sum.to_dataframe(name = 'runoff')
        # Add geometry ID
        df_t[geog_id] = p
        # Step 3.4 housekeeping
        del geom_sum
        return df_t
    

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    # Step 1 - Read in 5 arcmin dataset (steps are different for irrigation)
    runoff = find_runoff_paths(gcm, scen, m)
    ds_ww = read_NETCDF(runoff) 
    ww_band = 'land_surface_runoff'
    print(" - - - - Step 1: Read in runoff") 
    memory_usage()   
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #     
    # Step 2 - Convert flux to volume  
    stime = time.time()
    ds_vol = flux_to_volume(ds_ww, ww_band)
    del ds_ww
    # 5. Persist and Fill NA with hand-made function 
    ds_vol = ds_vol.persist()
    ds_vol = fillnas(ds_vol['volume'])
    print(' - - - - Step 2: Convertered flux to volume in {}'.format(time.time()-stime))
    memory_usage()
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    # Step 3 - Resample data
    stime = time.time()
    # 6. Resample data    
    ds_rs = resample_xarray(ds_vol, resample_size)
    del ds_vol
    print('- - - - Step 3: Resampled data in {}'.format(time.time()-stime))
    memory_usage()
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    # Step 4 - Chunk data
    stime = time.time()
    # Chunk data to improve speed
    ds_rs = ds_rs.persist()
    rechunked = ds_rs.chunk({"lon": 100, "lat": 100})
    # Fill NA's with where statement. fillna functions aren't working great
    rechunked2 = fillnas(rechunked)
    del ds_rs
    del rechunked
    print('- - - - Step 4: Chunked in {}'.format(time.time()-stime))
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    # Step 5 - Find list of regional pfafs
    geogidlookupPATH = '/dbfs/mnt/pgb-data-lake/aqueduct_dev/aux-boundaries/m_region-pfaf6-lookups/{0}_pfaf6_lookup.csv'.format
    df_pfs = pd.read_csv(geogidlookupPATH(m))
    pfs = df_pfs['pfaf_id'].tolist()
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    # Step 6 - Clip by polygons
    print(" - - Start Step 5: Start clip loop") 
    memory_usage()
    stime = time.time()
    # Set number of workers
    n_workers = 40
    # Keep track of while loop
    worker_count = 0
    # While catchments remain in objectids, repeat this process. For every new round, lessen the number of workers to prevent memory overload
    while len(pfs) > 0:
        worker_count += 1
        print('- - - - - run number', 1, "\n- - - - - - Remaining catchments:", len(pfs))
        # Segment catchments by workers for parallel process
        objectids_list = segment_id_list(lst=pfs, n= int(n_workers / worker_count))
        # Step 3. Clip and sum by polygon
        df_fs = []
        for oids in objectids_list:
            memory_usage()
            df_ts = Parallel(n_jobs=n_workers)(delayed(clip_by_pfaf_id)(p) for p in oids)
            df_t_all = pd.concat(df_ts)
            # Pull list of finished PFs, remove from overall list
            finished_pfs = list(set(df_t_all[geog_id].tolist()))
            pfs = [x for x in pfs if (x not in finished_pfs)]
            del df_ts
            df_fs.append(df_t_all)
            del df_t_all
            print("- - - - Remaining catchments:", len(pfs))
            gc.collect()
    df_f = pd.concat(df_fs)
    outPATH = newROOT(resample_size, gcm, scen) + newNAME(m)
    df_f.to_csv(outPATH)

    del df_f
    del df_fs
    gc.collect()

# Run

In [0]:
rs_size = 5
# # Full run for 1 Region
for gcm in gcmFolders:
    print("- ", gcm)
    for scen in scenFolders:
        if gcm == 'gswp3-w5e5' and scen != 'historical':
            continue
        else:
            print("- - ", scen)
            for m in tqdm(mFolders):
                print("- - -", m)
                if os.path.exists(newROOT(rs_size, gcm, scen) + newNAME(m)):
                    continue
                else:
                    run_runoff_parallelized(gcm = gcm, scen = scen, m = m, resample_size = rs_size)

- gfdl-esm4
- - ssp585
 0%| | 0/53 [00:00<?, ?it/s]- - - M0000001
 - - - - Step 1: Read in runoff
- - - Current memory usage is: 533.015625
- - - Conversion to volume (million m3) started
- - - Current memory usage is: 533.01953125
- - - - total area (m2): 6130056000000.0
- - - - total area (Million m2): 6130056.0
- - - - total WW in Million m3: 6479730.0
- - - Conversion complete.
- - - Current memory usage is: 854.84765625
 - - - - Step 2: Convertered flux to volume in 2.8754546642303467
- - - Current memory usage is: 854.84765625
- - - - Step 3: Resampled data in 32.14625549316406
- - - Current memory usage is: 8717.78515625
- - - - Step 4: Chunked in 8.771951913833618
 - - Start Step 5: Start clip loop
- - - Current memory usage is: 8718.03515625
- - - - - run number 1 
- - - - - - Remaining catchments: 406
- - - Current memory usage is: 8718.03515625
- - - - Remaining catchments: 366
- - - Current memory usage is: 8647.62890625
- - - - Remaining catchments: 326
- - - Current memory usage is: 8647.5078125
- - - - Remaining catchments: 286
- - - Current memory usage is: 8647.671875
- - - - Remaining catchments: 246
- - - Current memory usage is: 8647.52734375
- - - - Remaining catchments: 206
- - - Current memory usage is: 8647.68359375
- - - - Remaining catchments: 166
- - - Current memory usage is: 8647.53515625
- - - - Remaining catchments: 126
- - - Current memory usage is: 8647.6875
- - - - Remaining catchments: 86
- - - Current memory usage is: 8647.55078125
- - - - Remaining catchments: 46
- - - Current memory usage is: 8647.703125
- - - - Remaining catchments: 6
- - - Current memory usage is: 8647.55078125
- - - - Remaining catchments: 0
 2%|▏ | 1/53 [14:21<12:26:30, 861.37s/it]- - - M0000002
 - - - - Step 1: Read in runoff
- - - Current memory usage is: 8648.3984375
- - - Conversion to volume (million m3) started
- - - Current memory usage is: 8648.3984375
- - - - total area (m2): 7169712000000.0
- - - - total area (Million m2): 7169711.0
- - - - total WW in Million m3: 31646806.0
- - - Conversion complete.
- - - Current memory usage is: 8995.34375
 - - - - Step 2: Convertered flux to volume in 3.55747127532959
- - - Current memory usage is: 8995.34375
- - - - Step 3: Resampled data in 44.717864751815796
- - - Current memory usage is: 17668.83203125
- - - - Step 4: Chunked in 10.192875146865845
 - - Start Step 5: Start clip loop
- - - Current memory usage is: 17668.83203125
- - - - - run number 1 
- - - - - - Remaining catchments: 387
- - - Current memory usage is: 17668.83203125
- - - - Remaining catchments: 347
- - - Current memory usage is: 9498.72265625
- - - - Remaining catchments: 307
- - - Current memory usage is: 9498.71875
- - - - Remaining catchments: 267
- - - Current memory usage is: 9498.71484375
- - - - Remaining catchments: 227
- - - Current memory usage is: 9498.703125
- - - - Remaining catchments: 187
- - - Current memory usage is: 9498.703125
- - - - Remaining catchments: 147
- - - Current memory usage is: 9498.703125
- - - - Remaining catchments: 107
- - - Current memory usage is: 9498.6953125
- - - - Remaining catchments: 67
- - - Current memory usage is: 9498.71875
- - - - Remaining catchments: 27
- - - Current memory usage is: 9498.6953125
- - - - Remaining catchments: 0
 4%|▍ | 2/53 [28:06<11:54:15, 840.30s/it]- - - M0000003
 - - - - Step 1: Read in runoff
- - - Current memory usage is: 9506.15625
- - - Conversion to volume (million m3) started
- - - Current memory usage is: 9506.15625
- - - - total area (m2): 12052874000000.0
- - - - total area (Million m2): 12052875.0
- - - - total WW in Million m3: 3847812.2
- - - Conversion complete.
- - - Current memory usage is: 10132.015625
 - - - - Step 2: Convertered flux to volume in 6.274010896682739
- - - Current memory usage is: 10132.015625
- - - - Step 3: Resampled data in 67.9065248966217
- - - Current memory usage is: 25781.83203125
- - - - Step 4: Chunked in 22.95861530303955
 - - Start Step 5: Start clip loop
- - - Current memory usage i

In [0]:
runoff = find_runoff_paths('gfdl-esm4', 'ssp126', 'M0000001')
ds_ww = read_NETCDF(runoff) 

In [0]:
ds_ww['land_surface_runoff']

Out[8]:

<xarray.DataArray 'land_surface_runoff' (time: 1032, lat: 276, lon: 288)>
[82031616 values with dtype=float32]
Coordinates:
 * time (time) datetime64[ns] 2015-01-31 2015-02-28 ... 2100-12-31
 * lat (lat) float32 -12.04 -12.12 -12.21 ... -34.79 -34.88 -34.96
 * lon (lon) float32 11.04 11.12 11.21 11.29 ... 34.79 34.88 34.96
 spatial_ref int64 0
Attributes:
 standard_name: land_surface_runoff
 long_name: land_surface_runoff
 units: m.month-1 xarray.DataArray 'land_surface_runoff' time : 1032 lat : 276 lon : 288 ... [82031616 values with dtype=float32] Coordinates: (4) time (time) datetime64[ns] 2015-01-31 ... 2100-12-31 standard_name : time long_name : Days since 1901-01-01 array(['2015-01-31T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
 '2015-03-31T00:00:00.000000000', ..., '2100-10-31T00:00:00.000000000',
 '2100-11-30T00:00:00.000000000', '2100-12-31T00:00:00.000000000'],
 dtype='datetime64[ns]') lat (lat) float32 -12.04 -12.12 ... -34.88 -34.96 long_name : latitude units : degrees_north standard_name : latitude array([-12.041667, -12.125 , -12.208333, ..., -34.791668, -34.875 ,
 -34.958332], dtype=float32) lon (lon) float32 11.04 11.12 11.21 ... 34.88 34.96 standard_name : longitude long_name : longitude units : degrees_east array([11.041667, 11.125 , 11.208333, ..., 34.791668, 34.875 , 34.958332],
 dtype=float32) spatial_ref () int64 0 crs_wkt : GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] semi_major_axis : 6378137.0 semi_minor_axis : 6356752.314245179 inverse_flattening : 298.257223563 reference_ellipsoid_name : WGS 84 longitude_of_prime_meridian : 0.0 prime_meridian_name : Greenwich geographic_crs_name : WGS 84 grid_mapping_name : latitude_longitude spatial_ref : GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] array(0) Attributes: (3) standard_name : land_surface_runoff long_name : land_surface_runoff units : m.month-1